In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from math import sqrt

from sklearn.model_selection import train_test_split
# --- Model ---
from sklearn.linear_model import LinearRegression
# --- Evaluation ---
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import SCORERS
from sklearn import ensemble
from sklearn.model_selection import KFold

housedf= pd.read_csv("kc_house_data.csv")
housedf['yr_sold']=housedf['date'].str[:4].astype(int)

# drop the columns having bedrooms = 0 , bathrooms =0, yr_sold < yr_build
print(housedf[(housedf['yr_sold']<housedf['yr_built'])].index)
print(housedf[(housedf['bedrooms']==0)].index)
print(housedf[(housedf['bathrooms']==0)].index)

housedf= housedf.drop(housedf[(housedf['yr_sold']<housedf['yr_built'])].index)
housedf= housedf.drop(housedf[(housedf['bedrooms']==0)].index)
housedf= housedf.drop(housedf[(housedf['bathrooms']==0)].index)

print(housedf[(housedf['yr_sold']<housedf['yr_built'])].index)
print(housedf[(housedf['bedrooms']==0)].index)
print(housedf[(housedf['bathrooms']==0)].index)

# to check null values 
missing_count = housedf.isnull().sum()/len(housedf) *100
missing_count= missing_count[missing_count>0]
#print(missing_count.sort_values)


#housedf.info()
## plot to see the distribution
#sns.distplot(housedf['price'])
target=np.log(housedf['price'])
#sns.distplot(target)
#sns.heatmap(housedf.isnull(),yticklabels=False,cbar=False)


## Check numerical and catogorical columns

numerical_data=housedf.select_dtypes(include=[np.number])
catagorical_data=housedf.select_dtypes(exclude=[np.number])

# print(numerical_data.shape[1])
# print(catagorical_data.shape[1])



# correlation_matrix = housedf.corr().round(2)
# # annot = True to print the values inside the square
# sns.heatmap(data=correlation_matrix, annot=True)
# plt.show()

# sqft living is highly co related with price , 
# sqft living with highly corelated with other attribute sqft_above  = 0.88 so we can take only one 
# sqft_above is corelated with grade

# price code has negative highest co relation with price -0.05
# low co relation is condition with price

# plt.scatter(housedf['sqft_above'], housedf['price'], marker='o')
# plt.title('price Vs sqft_above')
# plt.xlabel('sqft_above')
# plt.ylabel('price1')
# #plt.show

# plt.scatter(housedf['grade'], housedf['price'], marker='o')
# plt.title('grade Vs sqft_above')
# plt.xlabel('grade')
# plt.ylabel('price2')
#plt.show


# r2 = r2_score(Y_train, y_train_predict)
# print("The model performance for training set")
# print("--------------------------------------")
# print('RMSE is {}'.format(rmse))
# print('R2 score is {}'.format(r2))
# print("\n")

# # model evaluation for testing set
# y_test_predict = lin_model.predict(X_test)
# rmse = (np.sqrt(mean_squared_error(Y_test, y_test_predict)))
# r2 = r2_score(Y_test, y_test_predict)
# print("The model performance for testing set")
# print("--------------------------------------")
# print('RMSE is {}'.format(rmse))
# print('R2 score is {}'.format(r2))

#print(sklearn.metrics.SCORERS.keys())

# Spot Check Algorithms


feature_names = ['sqft_living', 'grade','sqft_above','sqft_living15']
X = housedf[feature_names]
Y = housedf['price']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size = 0.2)

lin_model = LinearRegression()
lin_model.fit(X_train, Y_train)
y_predicted = lin_model.predict(X_test)
rms = sqrt(mean_squared_error(Y_test, y_predicted))
#print("Linear regression MSE: %.4f" % rms)

params_gd={
    'n_estimators': 300,
    'max_depth': 4,
    'learning_rate': 0.01,
    'min_samples_split': 2,
    'criterion': 'mse'
}

gd_boost=ensemble.GradientBoostingRegressor(**params_gd)

gd_boost.fit(X_train, Y_train)

y_predicted = gd_boost.predict(X_test)  #233878.66270215836

rms = sqrt(mean_squared_error(Y_test, y_predicted))

#print("Gradient boost MSE: %.4f" % rms)

#SVC(gamma='auto')


models = []
#models.append(('LOR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LR', LinearRegression()))
#models.append(('KNN', KNeighborsClassifier(n_neighbors=3)))
models.append(('Gradient Boost',ensemble.GradientBoostingRegressor(**params_gd)))
# models.append(('SVM', SVC(gamma='auto')))

results = []
names = []


for name, model in models:
    kfold = KFold(n_splits=5, random_state=1)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold)
    results.append(cv_results.mean())
    names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

from scipy.stats import ttest_ind
# write down your conclusion here (2 points)
LR_result = results[0]
GD_result= results[1]
alpha = 0.05

print(results[0], results[1])

stat, p = ttest_ind(LR_result, GD_result)
print(stat,p)

# rd_forest=ensemble.RandomForestRegressor(**params_rd)

# rd_forest.fit(X_train, Y_train)

# y_predicted = rd_forest.predict(X_test)  #233878.66270215836

# LR= LinearRegression()

# LR.fit(X_train, Y_train)

# y_predicted = LR.predict(X_test)   #248082.06970963092




    




Int64Index([1763, 2687, 7526, 8039, 14489, 17098, 19805, 20770, 20852, 20963,
            21262, 21372],
           dtype='int64')
Int64Index([875, 3119, 3467, 4868, 6994, 8477, 8484, 9773, 9854, 12653, 14423,
            18379, 19452],
           dtype='int64')
Int64Index([875, 1149, 3119, 5832, 6994, 9773, 9854, 10481, 14423, 19452], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
LR: 0.537306 (0.007001)
Gradient Boost: 0.593665 (0.023308)
0.5373064743884411 0.5936645186987269
nan nan


/home/teju2019/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/home/teju2019/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
